# CHRONNOS - automatic coronal hole detection

![](https://github.com/RobertJaro/MultiChannelCHDetection/blob/main/images/title.jpg?raw=true)

This notebook provides coronal hole detections for arbitrary dates. Use the form below to specify the observation date. The data is automatically downloaded from JSOC, afterwards CHRONNOS scans the files and provides the CH maps as FITS files. This version uses only 4 EUV channels for detecting coronal holes. The notebook uses online resources for the computations. You can download the results or continue your evaluation in this notebook.

(Predictions require a few seconds per sample. If you need to detect coronal holes from a larger data set you can activate the GPU acceleration to speed up the detections.)

## Install and imports

In [ ]:
!pip install matplotlib==3.1.3
!pip install sunpy==3.1.0
!pip install chronnos

In [ ]:
# imports for download
import os
import shutil
import drms
from datetime import datetime

# CHRONNOS detection tool
from chronnos.evaluate.detect import CHRONNOSDetector
from chronnos.data.convert import get_intersecting_files
from tqdm import tqdm

# visualization
from sunpy.map import Map
from matplotlib import pyplot as plt
from astropy.visualization import ImageNormalize, AsinhStretch
from sunpy.visualization.colormaps import cm

# file zip
import gzip
import glob

## Download data

In [ ]:
#@title Download Settings
download_dir = 'chronnos_series' #@param {type:"string"}

wavelengths = ['171', '193', '211', '304'] #@param {type:"raw"}

#@markdown Downloading data requires an active registration at JSOC. http://jsoc.stanford.edu/ajax/register_email.html (free of charge)
email = '\u003C\u003Cyour email>>' #@param {type:"string"}

# initialize the download client and directories
[os.makedirs(os.path.join(download_dir, wl), exist_ok=True) for wl in wavelengths]
client = drms.Client(email=email, verbose=True)

Use the JSOC notation to specify the duration and cadence. (e.g., `m=minutes, h=hours, d=days`)

In [ ]:
#@title Select observation series
year = 2022 #@param {type:"integer"}
month = 9 #@param {type:"integer"}
day = 28 #@param {type:"integer"}
hour = 0 #@param {type:"integer"}
minute = 0 #@param {type:"number"}
duration = '1d' #@param {type:"string"}
cadence = '3h' #@param {type:"string"}

# create datetime object
date = datetime(year, month, day, hour, minute)

The data download might take a few minutes, depending on the size of the requested series.

In [ ]:
r = client.export('aia.lev1_euv_12s[%s/%s@%s][%s]{image}' % (date.isoformat('T'), duration, cadence, ','.join(wavelengths)), )
r.wait()

downloaded_files = r.download(download_dir)
for f in downloaded_files.download:
  path_elements = os.path.basename(f).split('.')
  f_date = path_elements[2]
  wl = path_elements[3]
  shutil.move(f, os.path.join(download_dir, wl, f_date[:-1] + '.fits'))

## Detect coronal holes

In [ ]:
chronnos_detector = CHRONNOSDetector(model_name='chronnos_euv_v1_0.pt')

In [ ]:
# create dir for results
os.makedirs(os.path.join(download_dir, 'masks'), exist_ok=True)
os.makedirs(os.path.join(download_dir, 'imgs'), exist_ok=True)
# group FITS files
dirs = ['171', '193', '211', '304']
map_paths = get_intersecting_files(download_dir, dirs=dirs)
# plotting
norm = ImageNormalize(vmin=0, vmax=8000, stretch=AsinhStretch(0.005))
cmap = cm.sdoaia193

In [ ]:
# start detection
for ch_map, aia_map_path in tqdm(zip(chronnos_detector.ipredict(map_paths, reproject=False), 
                                     map_paths[1]), 
                                total=len(map_paths[3])):
  # save fits
  mask_path = os.path.join(download_dir, 'masks', os.path.basename(aia_map_path))
  Map(ch_map.data.astype('int16'), ch_map.meta).save(mask_path)
  # zip fits file
  with open(mask_path, 'rb') as f_in, gzip.open(mask_path + '.gz', 'wb') as f_out:
    f_out.writelines(f_in)
  os.remove(mask_path)
  # plot overlay (other evaluations can be also done here)
  aia_map = Map(aia_map_path)
  plt.figure(figsize=(10, 10))
  plt.subplot(111, projection=aia_map)
  plt.imshow(aia_map.data / aia_map.exposure_time.value, norm=norm, cmap=cmap)
  plt.title(aia_map.date)
  plt.xlabel('Helioprojective Longitude')
  plt.ylabel('Helioprojective Latitude')
  ch_map.draw_contours(levels=[0.5], colors=['red'])
  plt.savefig(os.path.join(download_dir, 'imgs', '%s.jpg' %ch_map.date.to_datetime().isoformat('T')))
  plt.close()

The results can be found in the menu on the left (select Files and then navigate to the `chronnos_series` directory).

For an easier download we can zip the masks.

In [ ]:
shutil.make_archive(os.path.join(download_dir, 'masks'), 'zip', os.path.join(download_dir, 'masks'))